<a href="https://colab.research.google.com/github/JulianaFanni/mvp-MachineLearning-Analytics/blob/main/Juliana_Fanni_MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP- Juliana Fanni

Disciplina: Sprint: Machine Learning & Analytics

Profs. Hugo Villamizar e Patrick Happ

Aluna: Juliana Cardozo Pereira Fanni

## 1. Definição do Problema

A instituição é uma organização que tem como objetivo projetos de pesquisas científicas na área de saúde.

Dentre das diversas iniciativas da instituição, temos uma Plataforma API (Assistência & Pesquisa Integradas) que integra a comunidade médica da Oncologia para desenvolver novas pesquisas com o instituto.
Contando com uma base unificada de dados clínicos integrada aos banco de dados, para desenvolver estudos que preencham lacunas importantes do conhecimento em oncologia. A missão da Plataforma API é fomentar o avanço da pesquisa em oncologia no Brasil a partir de um modelo inovador de colaboração no qual os médicos responsáveis pelo cuidado também assumem o papel de pesquisadores.

Diante do acima exposto, o objetivo deste trabalho é rastrear e identificar tipos de sinais suspeitos de cancer de pele do paciente e assim classificar como um Cancer contribuindo para o diagnóstico do médico.
Esta iniciativa vem se mostrado relevante pela possibilidade da identificação precoce de doenças que apresentam maior probabilidade de cura quando tratadas em estágios iniciais

O dataset usado neste projeto será do https://huggingface.co/datasets/marmal88/skin_cancer , por haver um grande numero de imagens dos diversos tipo de manchas na pele e servir de experimento para o dataset futuro do banco de dados da instituição.

In [31]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn.metrics as skm
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from IPython.display import Image, display
from sklearn.linear_model import LinearRegression
from datetime import datetime
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import numpy as np
import sklearn.metrics as skm
import matplotlib.pyplot as plt
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from google.colab import drive # acesso ao drive do google Colab - 'drive.mount'

from tensorflow.keras.preprocessing import image

In [32]:
# instalando a biblioteca gdown
!pip install gdown
import gdown

In [33]:
# instalando dependencias
! pip install transformers[torch] datasets evaluate scikit-learn matplotlib


In [34]:
from datasets import load_dataset

dataset = load_dataset("marmal88/skin_cancer")

In [35]:
#Analisando o Dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'image_id', 'lesion_id', 'dx', 'dx_type', 'age', 'sex', 'localization'],
        num_rows: 9577
    })
    validation: Dataset({
        features: ['image', 'image_id', 'lesion_id', 'dx', 'dx_type', 'age', 'sex', 'localization'],
        num_rows: 2492
    })
    test: Dataset({
        features: ['image', 'image_id', 'lesion_id', 'dx', 'dx_type', 'age', 'sex', 'localization'],
        num_rows: 1285
    })
})

O dataset já veio separado com as bases train, test e validation.
Agora, irei analisar o número de imagens que o dataset possui

In [36]:
train_dataset = len(dataset['train'])
val_dataset = len(dataset['validation'])
test_dataset = len(dataset['test'])

tam_train_dataset = len(dataset['train'])
tam_val_dataset = len(dataset['validation'])
tam_test_dataset = len(dataset['test'])

print(f"Temos um total de {tam_test_dataset} para o teste do modelo de deep learning")
print(f"Temos um total de {tam_train_dataset} para o treinamento do modelo de deep learning")
print(f"Temos um total de {tam_val_dataset} para avaliação do modelo de deep learning")


Temos um total de 1285 para o teste do modelo de deep learning
Temos um total de 9577 para o treinamento do modelo de deep learning
Temos um total de 2492 para avaliação do modelo de deep learning


In [37]:
print(f"Informações dos 10 primeiros de avaliação: \n {dataset['test'][:10]}")

print(f"Informações dos 10 primeiros de treinamento: \n {dataset['train'][:10]}")

print(f"Informações dos 10 primeiros de avaliação: \n {dataset['validation'][:10]}")




Informações dos 10 primeiros de avaliação: 
 {'image': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x450 at 0x7E613211B040>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x450 at 0x7E613211B0A0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x450 at 0x7E613211B160>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x450 at 0x7E6132118130>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x450 at 0x7E613211A980>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x450 at 0x7E613211A920>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x450 at 0x7E6132118190>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x450 at 0x7E6132118280>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x450 at 0x7E613211B610>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x450 at 0x7E613211B4F0>], 'image_id': ['ISIC_0024329', 'ISIC_0024654', 'ISIC_0024707', 'ISIC_0024763', 'ISIC_0025539', 'ISIC_0025605', 'ISI

## 2. Acessando e tratando os dados que serão a entrada do modelo de deep learning

In [38]:
# Dividindo o dataset em tamanho menor, por muitas vezes dar erro no código por não conseguir executar por falta de memoria
subset_dataset_train = dataset['train'].select(range(4000))
subset_dataset_teste = dataset['test'].select(range(1000))
subset_dataset_aval = dataset['validation'].select(range(1000))


In [39]:
tam_test_dataset = len(subset_dataset_teste)
tam_train_dataset = len(subset_dataset_train)
tam_aval_dataset = len(subset_dataset_aval)

print(f"Temos um total de {tam_test_dataset} para o teste do modelo de deep learning")
print(f"Temos um total de {tam_train_dataset} para o treinamento do modelo de deep learning")
print(f"Temos um total de {tam_aval_dataset} para o avaliação do modelo de deep learning")



Temos um total de 1000 para o teste do modelo de deep learning
Temos um total de 4000 para o treinamento do modelo de deep learning
Temos um total de 1000 para o avaliação do modelo de deep learning


## Preparação dos dados

In [40]:
# Convertendo para um DataFrame pandas o dataset de Treinamento
df_train = pd.DataFrame(subset_dataset_train, columns=subset_dataset_train.features)
df_train['class'] = 0
df_train.head()

,image,image_id,lesion_id,dx,dx_type,age,sex,localization,class
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,ISIC_0024329,HAM_0002954,actinic_keratoses,histo,75.0,female,lower extremity,0
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,ISIC_0024372,HAM_0005389,actinic_keratoses,histo,70.0,male,lower extremity,0
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,ISIC_0024418,HAM_0003380,actinic_keratoses,histo,75.0,female,lower extremity,0
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,ISIC_0024450,HAM_0005505,actinic_keratoses,histo,50.0,male,upper extremity,0
4,<PIL.JpegImagePlugin.JpegImageFile image mode=...,ISIC_0024463,HAM_0004568,actinic_keratoses,histo,50.0,male,upper extremity,0


In [ ]:
# Convertendo para um DataFrame pandas o dataset de Teste
df_teste = pd.DataFrame(subset_dataset_teste, columns=subset_dataset_teste.features)
df_teste['class'] = 0
df_teste.head()

In [ ]:
# Convertendo para um DataFrame pandas o dataset de aval
df_aval = pd.DataFrame(subset_dataset_aval, columns=subset_dataset_teste.features)
df_aval['class'] = 0
df_aval.head()

In [ ]:
def calcula_classe_cancer(cancer):
  vcancer = cancer
  if cancer == 'melanoma' or cancer == 'basal_cell_carcinoma' or cancer == 'melanocytic_Nevi' :
    return "Cancer Positivo"
  else :
     return "Cancer Negativo"

# Função que atribuiu a variável "class" com valores categóricos
def define_classe_cancer(cancer):
  vcancer = cancer
  if cancer == 'melanoma' or cancer == 'basal_cell_carcinoma' or cancer == 'melanocytic_Nevi' :
    return 1
  else :
    return 0


In [ ]:
# Atribuiu a variável "class" com valores categóricos no dataset
for i in range(tam_test_dataset):
  vclasse = define_classe_cancer(df_teste['dx'][i])
  df_teste.loc[i,'class'] = (vclasse)

for i in range(tam_train_dataset):
  vclasse = define_classe_cancer(df_train['dx'][i])
  df_train.loc[i,'class'] = (vclasse)

for i in range(tam_aval_dataset):
  vclasse = define_classe_cancer(df_aval['dx'][i])
  df_aval.loc[i,'class'] = (vclasse)

In [ ]:
# Definação do tamanho do batch e as dimensões das imagens
batch_size = 32
img_height = 224
img_width = 224
num_classes = 5

In [ ]:
#Visualização do número de imagens por classe
print((df_teste['dx']).value_counts())

plt.figure(figsize=(18, 5))
fire_count = df_teste['dx'].value_counts().plot.bar(title='Dados disponíveis Imagens de Testes por tipo')
plt.subplots_adjust(top=0.9)
plt.xticks(rotation=360)
plt.show()

In [ ]:
#Visualização do número de imagens por classe no dataset de teste
print((df_teste['class']).value_counts())

plt.figure(figsize=(18, 5))
fire_count = df_teste['class'].value_counts().plot.bar(title='Dados disponíveis Imagens de Testes - Positivp x Negativo')
plt.subplots_adjust(top=0.9)
plt.xticks(rotation=360)
plt.tight_layout()
plt.ylabel('Quantidade')
plt.xlabel('Cancer')
plt.show()

In [ ]:
#Visualização do número de imagens por classe no dataset de treinamento
print((df_train['dx']).value_counts())

plt.figure(figsize=(18, 5))
fire_count = df_train['dx'].value_counts().plot.bar(title='Dados disponíveis Imagens de Treinamento')
plt.subplots_adjust(top=0.9)
plt.xticks(rotation=360)
plt.show()

In [ ]:
#Visualização do número de imagens por classe no dataset de teste
print((df_train['class']).value_counts())

plt.figure(figsize=(18, 5))
fire_count = df_train['class'].value_counts().plot.bar(title='Dados disponíveis Imagens de Treinamento - Positivp x Negativo')
plt.subplots_adjust(top=0.9)
plt.xticks(rotation=360)
plt.tight_layout()
plt.ylabel('Quantidade')
plt.xlabel('Cancer')
plt.show()

In [ ]:
images = df_teste['image']
print(images)

In [ ]:
df_teste['class'] = df_teste['class'].astype('str')
df_teste['image'] = df_teste['image'].astype('str')

fig = plt.gcf()
fig.set_size_inches(3*4, 3*4)

# Visualizando apenas 12 imagens Teste
for i in range(12):
    sp = plt.subplot(3, 4, i + 1)
    sp.axis('Off')
    img_train = dataset["test"][i]['image'].resize((img_height, img_width))
    verifica_positivo=calcula_classe_cancer(dataset["test"][i]['dx'])
    plt.title(verifica_positivo)
    plt.imshow(img_train)

print(f"Exemplos de 12 imagens de Teste: \n")
plt.axis('off')  # Para não exibir os eixos
plt.show()

In [ ]:
fig = plt.gcf()
fig.set_size_inches(3*4, 3*4)

# Visualizando apenas 12 imagens Treinamento
for i in range(12):
    sp = plt.subplot(3, 4, i + 1)
    sp.axis('Off')
    img_train = dataset["train"][i]['image'].resize((img_height, img_width))
    verifica_positivo=calcula_classe_cancer(dataset["test"][i]['dx'])
    plt.title(verifica_positivo)
    plt.imshow(img_train)

print(f"Exemplos de 12 imagens de Treinamento: \n")
plt.axis('off')  # Para não exibir os eixos
plt.show()

## 4. Treinamento do modelo de deep learning

In [ ]:
path = 'datasets/train'

train_dir = 'datasets/train'
test_dir = 'datasets/test'
val_dir = 'datasets/aval'

train_datagen=ImageDataGenerator(validation_split=0.2, #include validation split
                                 rescale = 1.0/255,
                                    #rotation_range=40,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=train_datagen.flow_from_dataframe(
    df_teste ,train_dir,
    target_size=(150,150),
    batch_size=100, #50
    class_mode='binary',
    x_col='image',
    y_col='class',
    subset='training' #training set
    )

val_generator=train_datagen.flow_from_dataframe(
    df_teste ,val_dir,
    target_size=(150,150),
    batch_size=100, #50
    class_mode='binary',
    x_col='image',
    y_col='class',
    subset='validation' #validation set
    )

test_generator=test_datagen.flow_from_dataframe(
    df_teste ,test_dir,
    target_size=(150,150),
    batch_size=100, #50
    class_mode='binary',
    x_col='image',
    y_col='class'
    )

In [ ]:
model=models.Sequential()
model.add(layers.Conv2D(32 ,(3,3) , activation='relu' , input_shape=(150,150,3 ) ) )
model.add(layers.MaxPooling2D( (2,2)))

model.add(layers.Conv2D(64 , (3,3) , activation='relu'))
model.add(layers.MaxPooling2D( (2,2)))

model.add(layers.Conv2D(128 , (3,3) , activation='relu'))
model.add(layers.MaxPooling2D( (2,2)))

model.add(layers.Conv2D(128 , (3,3) , activation='relu'))
model.add(layers.MaxPooling2D( (2,2)))



In [ ]:
model.add(layers.Flatten() )
model.add(layers.Dropout(0.25))
model.add(layers.Dense( 512 ,activation='relu' ))
model.add(layers.Dropout(0.25))
model.add(layers.Dense( 1 , activation='sigmoid' ) )

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy' , metrics=['accuracy'])

if len(train_generator) > 0   :
      history=model.fit_generator(train_generator,
                                  epochs=20,
                                  validation_data=val_generator);

      # livro jornada python pag 493
      plt.xlabel('Época')
      plt.ylabel('Acurácia')
      plt.plot(history.history['accuracy'], '--')
      plt.plot(history.history['val_accuracy'])
      plt.title("Evolução da acurácia por épocas")
      plt.legend(['Treino', 'Validação'])
      plt.show();

5 - Conclusão

Neste MVP, foi o inicio de um projeto de grande potencial e com possibilidades de amadureciimento, crescimento e varias sprints para  diversas areas da medicina. No enatnto, objtive dificuldades iniciais por limitações na CPU do Colab, supernado esse aspecto é um projeto com grande possibilidade e idéias.